In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [3]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
df_train = pd.read_pickle('../datasets/nprint-raw/netml-iot/X.pkl')
df_test = pd.read_pickle('../datasets/nprint-raw/netml-iot/X_val.pkl')
label_train = pd.read_pickle('../datasets/nprint-raw/netml-iot/y.pkl')
label_test = pd.read_pickle('../datasets/nprint-raw/netml-iot/y_val.pkl')
# Save a copy as CSV
df_train.to_csv('../datasets/nprint-raw/netml-iot/X.csv')
df_test.to_csv('../datasets/nprint-raw/netml-iot/X_val.csv')
label_train.to_csv('../datasets/nprint-raw/netml-iot/y.csv')
label_test.to_csv('../datasets/nprint-raw/netml-iot/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/netml-iot/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/netml-iot/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/netml-iot/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/netml-iot/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/netml-iot/feature-importance.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

In [4]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")
display(features_rankings_df.head(40))

,pkt_0_ipv4_tos_0,pkt_0_ipv4_tos_2,pkt_0_ipv4_tos_5,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,...,pkt_9_udp_cksum_6,pkt_9_udp_cksum_7,pkt_9_udp_cksum_8,pkt_9_udp_cksum_9,pkt_9_udp_cksum_10,pkt_9_udp_cksum_11,pkt_9_udp_cksum_12,pkt_9_udp_cksum_13,pkt_9_udp_cksum_14,pkt_9_udp_cksum_15
0,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,0,0,0,0,1,0,1,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11983,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11984,0,0,0,0,0,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11985,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,pkt_0_ipv4_tos_0,pkt_0_ipv4_tos_2,pkt_0_ipv4_tos_5,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,...,pkt_9_udp_cksum_6,pkt_9_udp_cksum_7,pkt_9_udp_cksum_8,pkt_9_udp_cksum_9,pkt_9_udp_cksum_10,pkt_9_udp_cksum_11,pkt_9_udp_cksum_12,pkt_9_udp_cksum_13,pkt_9_udp_cksum_14,pkt_9_udp_cksum_15
0,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,0,0,0,0,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,0,0,0,0,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1328,0,0,0,0,0,0,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1329,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1330,0,0,0,0,0,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,label
0,1
1,1
2,1
3,1
4,1
...,...
11982,1
11983,1
11984,0
11985,1


,label
0,1
1,1
2,1
3,0
4,0
...,...
1327,1
1328,1
1329,1
1330,1


label
1    8915
0    3072
Name: count, dtype: int64
label
1    991
0    341
Name: count, dtype: int64
df_train shape: (11987, 4506) 
 columns: ['pkt_0_ipv4_tos_0' 'pkt_0_ipv4_tos_2' 'pkt_0_ipv4_tos_5' ...
 'pkt_9_udp_cksum_13' 'pkt_9_udp_cksum_14' 'pkt_9_udp_cksum_15']
df_test shape: (1332, 4506) 
 columns: ['pkt_0_ipv4_tos_0' 'pkt_0_ipv4_tos_2' 'pkt_0_ipv4_tos_5' ...
 'pkt_9_udp_cksum_13' 'pkt_9_udp_cksum_14' 'pkt_9_udp_cksum_15']


,Unnamed: 0,importance,stddev,p_value,n,p99_high,p99_low
0,pkt_1_ipv4_dfbit_0,0.3028,0.006989,1.493670e-16,10,0.309982,0.295618
1,pkt_1_ipv4_ttl_1,0.1099,0.010692,6.072644e-11,10,0.120888,0.098912
2,pkt_1_ipv4_ttl_0,0.0044,0.000966,8.025068e-08,10,0.005393,0.003407
3,pkt_0_ipv4_ttl_1,0.0023,0.001337,2.060370e-04,10,0.003675,0.000925
4,pkt_3_ipv4_ttl_2,0.0013,0.000823,3.728090e-04,10,0.002146,0.000454
5,pkt_1_udp_dport_0,0.0012,0.000789,4.796551e-04,10,0.002011,0.000389
6,pkt_1_ipv4_tl_7,0.0010,0.000000,5.000000e-01,10,0.001000,0.001000
7,pkt_0_udp_len_9,0.0009,0.000568,3.626075e-04,10,0.001483,0.000317
8,pkt_1_ipv4_ver_0,0.0009,0.000316,4.269026e-06,10,0.001225,0.000575
9,pkt_0_ipv4_tl_10,0.0009,0.000316,4.269026e-06,10,0.001225,0.000575


In [5]:
#make a list with only the best features

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:16]
print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]

display(df_train)
display(df_test)

['pkt_1_ipv4_dfbit_0', 'pkt_1_ipv4_ttl_1', 'pkt_1_ipv4_ttl_0', 'pkt_0_ipv4_ttl_1', 'pkt_3_ipv4_ttl_2', 'pkt_1_udp_dport_0', 'pkt_1_ipv4_tl_7', 'pkt_0_udp_len_9', 'pkt_1_ipv4_ver_0', 'pkt_0_ipv4_tl_10', 'pkt_0_udp_len_14', 'pkt_1_tcp_dprt_0', 'pkt_0_tcp_doff_2', 'pkt_1_tcp_doff_2', 'pkt_2_tcp_opt_39', 'pkt_0_udp_len_6']


,pkt_1_ipv4_dfbit_0,pkt_1_ipv4_ttl_1,pkt_1_ipv4_ttl_0,pkt_0_ipv4_ttl_1,pkt_3_ipv4_ttl_2,pkt_1_udp_dport_0,pkt_1_ipv4_tl_7,pkt_0_udp_len_9,pkt_1_ipv4_ver_0,pkt_0_ipv4_tl_10,pkt_0_udp_len_14,pkt_1_tcp_dprt_0,pkt_0_tcp_doff_2,pkt_1_tcp_doff_2,pkt_2_tcp_opt_39,pkt_0_udp_len_6
0,1,1,0,1,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
1,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
2,1,0,0,0,0,0,0,0,0,1,0,-1,-1,-1,-1,0
3,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
4,1,1,0,1,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
11983,1,1,0,1,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
11984,0,1,0,0,0,1,0,0,0,1,0,-1,-1,-1,-1,0
11985,1,0,1,0,-1,-1,0,-1,0,1,-1,0,0,0,1,-1


,pkt_1_ipv4_dfbit_0,pkt_1_ipv4_ttl_1,pkt_1_ipv4_ttl_0,pkt_0_ipv4_ttl_1,pkt_3_ipv4_ttl_2,pkt_1_udp_dport_0,pkt_1_ipv4_tl_7,pkt_0_udp_len_9,pkt_1_ipv4_ver_0,pkt_0_ipv4_tl_10,pkt_0_udp_len_14,pkt_1_tcp_dprt_0,pkt_0_tcp_doff_2,pkt_1_tcp_doff_2,pkt_2_tcp_opt_39,pkt_0_udp_len_6
0,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
1,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
2,0,0,0,0,-1,0,0,0,0,1,1,-1,-1,-1,-1,0
3,0,1,0,0,0,1,0,0,0,1,0,-1,-1,-1,-1,0
4,0,1,0,0,-1,1,0,0,0,1,1,-1,-1,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,1,1,0,0,0,-1,0,-1,0,1,-1,1,1,1,-1,-1
1328,0,0,0,0,-1,1,0,0,0,0,1,-1,-1,-1,-1,0
1329,1,1,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,-1
1330,1,1,0,1,0,-1,0,-1,0,1,-1,1,0,0,-1,-1


In [6]:
# Model Parameters
number_of_attributes = df_train.shape[1]
hidden_layer_nodes = 32
output_layer_nodes = 3

# Traing Parameters
EPOCHS = 500
BATCH_SIZE = 500

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 10

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [7]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [8]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 4.0479e-04 
1: train_accuracy:[0.9998331665992737] test_accuracy:[1.0]
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step
tf.Tensor(
[[341   0]
 [  0 991]], shape=(2, 2), dtype=int32)
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 1.5115e-04 
2: train_accuracy:[0.9998331665992737, 0.9998331665992737] test_accuracy:[1.0, 1.0]
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step
tf.Tensor(
[[341   0]
 [  0 991]], shape=(2, 2), dtype=int32)
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 2.4977e-04 
3: train_accuracy:[0.9998331665992737, 0.9998331665992737, 0.9998331665992737] test_accuracy:[1.0, 1.0, 1.0]
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step
tf.Tensor(
[[341   0]
 [  0 991]], shape=(2, 2), dtype=int32)
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 3.5820e-04 
4: train_accuracy:[0.9998331665992737, 0.9998331665992737, 0.9998331665992737, 0.9998331665992737] test_accuracy:[1.0, 1.0, 1.0, 1.0]
4

In [9]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

20-06-2024-15-07-23
C:\NN-P4\tf-model-testing
